# Star Formation

In [13]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import savgol_filter

In [14]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration, add_redshift
from auriga.settings import Settings
from auriga.support import make_snapshot_number
from auriga.parser import parse

In [15]:
figure_setup()

In [16]:
settings = Settings()

In [17]:
def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
    """
    This method adds a panel with the results for a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation.
    ax : plt.Axes
        The ax to which to add the plot.
    """

    settings = Settings()

    df = pd.read_csv(f"../results/{simulation}/temporal_data.csv")
    df["SFR_Total_Msun/yr"] = df[
        ["SFR_H_Msun/yr", "SFR_B_Msun/yr",
         "SFR_CD_Msun/yr", "SFR_WD_Msun/yr"]].sum(axis=1)
    df.dropna(inplace=True)

    for component in settings.components:
        ax.plot(df["Time_Gyr"],
                savgol_filter(df[f"SFR_{component}_Msun/yr"], 5, 1),
                c=settings.component_colors[component],
                lw=1.0, label=settings.component_labels[component], zorder=10)
    ax.plot(df["Time_Gyr"], savgol_filter(df["SFR_Total_Msun/yr"], 5, 1),
            c='k', lw=1.0, label="Total", zorder=10)

    ax.text(x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + simulation.upper() + "}$",
            ha='right', va='top', transform=ax.transAxes)

In [18]:
def plot_sfr_by_region_for_sample(simulations: list, filename: str):
    """
    This method creates a plot of the star formation rate (SFR) by
    galactic component for each simulation.

    Parameters
    ----------
    simulations : str
        A list of simulations to plot.
    filename : str
        The name of the output file.
    """

    n_simulations = len(simulations)

    fig = plt.figure(figsize=(7.2, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    set_axs_configuration(
        xlim=(0, 14), ylim=(0.1, 20),
        xticks=[2, 4, 6, 8, 10, 12], yticks=[0.1, 1, 10],
        xlabel="Time [Gyr]",
        ylabel=r"SFR [$\mathrm{M}_\odot \, \mathrm{yr}^{-1}$]",
        axs=axs, n_used=n_simulations,
        yticklabels=[0.1, 1, 10], yscale="log",
        )

    for idx, ax in enumerate(axs.flat):
        if idx < n_simulations:
            add_panel_for_galaxy(simulations[idx], ax=ax)
        else:
            ax.axis("off")
        
        if ax == axs[0, 0]:
            ax.legend(loc="upper left", framealpha=0.0, fontsize=4.0)

        ax.xaxis.label.set_size(8.0)
        ax.yaxis.label.set_size(8.0)

    fig.savefig(f"../images/sfr_by_region/{filename}.pdf")
    plt.close(fig)

In [20]:
# Create plots for all galaxies
originals = [f"au{i}_or_l4" for i in settings.galaxies]
reruns = [f"au{i}_re_l4" for i in settings.reruns]
plot_sfr_by_region_for_sample(simulations=originals, filename="originals")
plot_sfr_by_region_for_sample(simulations=reruns, filename="reruns")